In [5]:
#!pip install -q -U sagemaker

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 2.19.1 requires aiohttp, which is not installed.
datasets 2.19.1 requires filelock, which is not installed.
datasets 2.19.1 requires fsspec[http]<=2024.3.1,>=2023.1.0, which is not installed.
datasets 2.19.1 requires pyarrow-hotfix, which is not installed.
datasets 2.19.1 requires xxhash, which is not installed.
huggingface-hub 0.23.1 requires filelock, which is not installed.
huggingface-hub 0.23.1 requires fsspec>=2023.5.0, which is not installed.
tensorflow-datasets 4.9.4 requires dm-tree, which is not installed.
tensorflow-datasets 4.9.4 requires termcolor, which is not installed.
tensorflow-datasets 4.9.4 requires toml, which is not installed.
transformers 4.41.1 requires filelock, which is not installed.
transformers 4.41.1 requires regex!=2019.12.17, which is not installed.
transformers 4.41.1 requ

In [2]:
#%load_ext nb_black

In [1]:
import os
import logging
from datetime import datetime

In [6]:
import boto3
import sagemaker
from sagemaker.session import TrainingInput
from sagemaker import image_uris
from sagemaker import hyperparameters

sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\pat\AppData\Local\sagemaker\sagemaker\config.yaml


In [7]:
boto3.set_stream_logger(name="botocore.credentials", level=logging.WARNING)

In [8]:
region = sagemaker.Session().boto_region_name
print(region)

us-east-1


In [8]:
# role_arn = sagemaker.get_execution_role()
#role_arn = os.getenv("SGMKR_ROLE_ARN")
role_arn = "arn:aws:iam::012345678901:role/service-role/AmazonSageMaker-ExecutionRole-20240718T104942"

In [9]:
bucket = "pat-demo-bkt"
prefix = "iris"

In [10]:
!aws s3 ls {bucket}/{prefix}/

                           PRE batch_transform/
                           PRE data/
2024-10-16 17:58:20          0 


In [11]:
!aws s3 ls {bucket}/{prefix}/data/ --recursive

2024-10-16 17:58:39          0 iris/data/
2024-10-16 17:59:10        950 iris/data/iris_test.csv
2024-10-16 17:59:07       1900 iris/data/iris_train.csv


In [13]:
train_file = "data/iris_train.csv"
valid_file = "data/iris_test.csv"

train_file_uri = "s3://{}/{}/{}".format(bucket, prefix, train_file)
valid_file_uri = "s3://{}/{}/{}".format(bucket, prefix, valid_file)
print("train file uri:", train_file_uri)
print("valid file uri:", valid_file_uri)

train file uri: s3://pat-demo-bkt/iris/data/iris_train.csv
valid file uri: s3://pat-demo-bkt/iris/data/iris_test.csv


In [17]:
train_ip = TrainingInput(train_file_uri, content_type="csv")
print(train_ip)

In [18]:
valid_ip = TrainingInput(valid_file_uri, content_type="csv")
print(valid_ip)

In [19]:
model_op = "s3://{}/{}/{}".format(bucket, prefix, "model")
print(model_op)

s3://pat-demo-bkt/iris/model


In [20]:
model_img = sagemaker.image_uris.retrieve("xgboost", region, "latest")
print(model_img)

811284229777.dkr.ecr.us-east-1.amazonaws.com/xgboost:latest


In [21]:
base_job_name = "iris-xgboost-"

In [22]:
xgb_model = sagemaker.estimator.Estimator(
    image_uri=model_img,
    role=role_arn,
    base_job_name=base_job_name,
    instance_count=1,
    instance_type="ml.m4.xlarge",
    output_path=model_op,
    sagemaker_session=sagemaker.Session(),
    volume_size=5,
)

In [23]:
xgb_model.set_hyperparameters(
    num_class=3, max_depth=5, num_round=10, objective="multi:softmax",
)

In [24]:
job_name = base_job_name + datetime.today().strftime("%Y-%m-%d-%H-%M-%S")
print(job_name)

iris-xgboost-2024-10-17-13-45-55


In [25]:
xgb_model.fit({"train": train_ip, "validation": valid_ip}, wait=True, job_name=job_name)

INFO:sagemaker:Creating training-job with name: iris-xgboost-2024-10-17-13-45-55


2024-10-17 20:46:26 Starting - Starting the training job...
2024-10-17 20:46:41 Starting - Preparing the instances for training...
2024-10-17 20:47:12 Downloading - Downloading input data...
2024-10-17 20:47:48 Downloading - Downloading the training image...
2024-10-17 20:48:33 Training - Training image download completed. Training in progress..Arguments: train
[2024-10-17:20:48:46:INFO] Running standalone xgboost training.
[2024-10-17:20:48:46:INFO] File size need to be processed in the node: 0.0mb. Available memory size in the node: 8461.59mb
[2024-10-17:20:48:46:INFO] Determined delimiter of CSV input is ','
[20:48:46] S3DistributionType set as FullyReplicated
[20:48:46] 100x4 matrix with 400 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[2024-10-17:20:48:46:INFO] Determined delimiter of CSV input is ','
[20:48:46] S3DistributionType set as FullyReplicated
[20:48:46] 50x4 matrix with 200 entries loaded from /opt/ml/input/data/validation?format=cs

In [26]:
!aws s3 ls {bucket}/{prefix}/model/

                           PRE iris-xgboost-2024-10-17-13-45-55/


In [27]:
!aws s3 ls {bucket}/{prefix}/model/{job_name}/

                           PRE debug-output/
                           PRE output/
                           PRE profiler-output/
